In [4]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt 
import math
from ipynb.fs.full.Utils import createKDeplot, loadDataFrames, dropFeaturesWithNoVariance,loadCoverGroups
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import stats
import dask_ml.cluster
import pandas as pd
import gower
from ipynb.fs.full.Pipeline import removeConstantFeatures, applyBoxCoxAllFeatures, centerAndScale, applyEuclideanDistance, getIndexAndVector, applyBoxCoxFeatures,applyJaccard, getEuclideanJaccardDf , applyCosineDistance

In [13]:

Xd=pd.DataFrame({'age':[21,21,19, 30,21,21,19,30,None],
'gender':['M','M','N','M','F','F','F','F',None],
'civil_status':['MARRIED','SINGLE','SINGLE','SINGLE','MARRIED','SINGLE','WIDOW','DIVORCED',None],
'salary':[3000.0,1200.0 ,32000.0,1800.0 ,2900.0 ,1100.0 ,10000.0,1500.0,None],
'has_children':[1,0,1,1,1,0,0,1,None],
'available_credit':[2200,100,22000,1100,2000,100,6000,2200,None]})

In [30]:
spotifyFeatures, transitionMatrices, df = loadDataFrames()
ids = transitionMatrices.index.tolist()
transitionMatrices['id'] = ids

In [32]:
df = transitionMatrices
df

,0,1,2,3,4,5,6,7,8,9,...,1286,1287,1288,1290,1291,1292,1293,1294,1295,id
id,,,,,,,,,,,,,,,,,,,,,
23,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,23
105,0.185185,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,105
118,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.044944,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,118
120,0.029412,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,120
292,0.013889,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.013889,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6002496,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,6002496
6002497,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,6002497
6002498,0.000000,0.015385,0.0,0.015385,0.0,0.015385,0.030769,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,6002498


In [34]:
ids

0            23
1           105
2           118
3           120
4           292
         ...   
5975    6002496
5976    6002497
5977    6002498
5978    6002499
5979    6002500
Name: id, Length: 5980, dtype: int64

In [33]:
## Removing unamed and other unnecessary columns
df.reset_index(drop=True, inplace=True)
df = df.sort_values('id')
df.id = df.id.astype('int64')
# Saving idChords
ids = df.id
df = df.set_index('id')
df.reset_index(drop=True, inplace=True)
df

,0,1,2,3,4,5,6,7,8,9,...,1284,1286,1287,1288,1290,1291,1292,1293,1294,1295
0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
1,0.185185,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
2,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.044944,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
3,0.029412,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
4,0.013889,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.013889,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5975,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
5976,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
5977,0.000000,0.015385,0.0,0.015385,0.0,0.015385,0.030769,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
5978,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.071429,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.02381,0.0,0.0,0.0,0.02381,0.0,0.0


In [35]:
XGower = np.asarray(df)
XGower

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.18518519, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.01538462, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.02380952, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
#pd.DataFrame(gower.gower_matrix(XGower))

In [38]:
pd.DataFrame(gower.gower_topn(df.loc[0:1,:], df.iloc[:,], n = 50))

,index,values
0,0,0.000000
1,1624,0.002520
2,1133,0.002590
3,3451,0.002934
4,619,0.003061
5,353,0.003075
6,1255,0.003108
7,1241,0.003175
8,847,0.003190
9,2604,0.003211


In [2]:
coverList = loadCoverGroups()